In [0]:
import requests
import base64
import time
from urllib.parse import urlparse, parse_qs, urlencode

In [0]:
CLIENT_ID = "9c22135f704248acb4c79520646559c3"
CLIENT_SECRET = "7d81b48def1e427580328769dd1349df"
REDIRECT_URI = "http://127.0.0.1:5000/callback"

# Define the permissions your pipeline needs
SCOPES = (
    "user-read-recently-played "
    "playlist-read-private "
    "user-library-read"
)

AUTH_URL = "https://accounts.spotify.com/authorize"

# Create the widget where you will paste the URL
dbutils.widgets.text("1_PASTE_URL_HERE", "", "1. Paste the URL from localhost here")

def generate_auth_url(client_id, redirect_uri, scopes, auth_url):
    auth_params = {
        "client_id": client_id,
        "response_type": "code",
        "redirect_uri": redirect_uri,
        "scope": scopes,
    }
    return f"{auth_url}?{urlencode(auth_params)}"

def print_instructions(auth_url):
    print("--- STEP 1: AUTHORIZATION ---")
    print("Copy the link below and open it in your local browser:")
    print("\n")
    print(auth_url)
    print("\n--- STEP 2: WAIT ---")
    print("Log in, accept the permissions, and you will be redirected to 'localhost' (which will show an error).")
    print("Copy the full URL from your browser's address bar and paste it into the '1_PASTE_URL_HERE' box above.")

auth_redirect_url = generate_auth_url(CLIENT_ID, REDIRECT_URI, SCOPES, AUTH_URL)
print_instructions(auth_redirect_url)

In [0]:
TOKEN_URL = "https://accounts.spotify.com/api/token"

# --- 1. Retrieve the URL from the widget ---
redirected_url = dbutils.widgets.get("1_PASTE_URL_HERE")

def get_auth_code(redirected_url):
    parsed_url = urlparse(redirected_url)
    return parse_qs(parsed_url.query).get('code', [None])[0]

def request_refresh_token(auth_code):
    auth_str = f"{CLIENT_ID}:{CLIENT_SECRET}"
    auth_b64 = base64.b64encode(auth_str.encode()).decode()
    headers = {"Authorization": f"Basic {auth_b64}"}
    data = {
        "grant_type": "authorization_code",
        "code": auth_code,
        "redirect_uri": REDIRECT_URI
    }
    response = requests.post(TOKEN_URL, data=data, headers=headers)
    response.raise_for_status()
    return response.json()

def print_success(refresh_token):
    print("\n" + "="*50)
    print("--- ✅ SUCCESS! REFRESH TOKEN OBTAINED ---")
    print("This is your new Refresh Token. Copy it.")
    print("\n")
    print(refresh_token)
    print("\n" + "="*50)
    print("\n--- NOW ---")
    print("1. Save this token in Databricks Secrets:")
    print(f"  databricks secrets put-secret spotify_secrets refresh-token --string-value=\"{refresh_token}\"")
    print("2. Delete this temporary notebook ('TEMP_Get_Token').")

if not redirected_url:
    print("ERROR: You did not paste the URL in the '1_PASTE_URL_HERE' box above.")
    print("Run Cell 2 again, follow the steps in your browser, paste the URL, then run this cell.")
else:
    try:
        # --- 2. Extract the authorization code ---
        auth_code = get_auth_code(redirected_url)
        if not auth_code:
            raise ValueError("Authorization code was not found in the URL.")
        print("Authorization code obtained successfully!")

        # --- 3. Exchange the code for a REFRESH TOKEN ---
        print("Requesting Refresh Token...")
        token_info = request_refresh_token(auth_code)

        if "refresh_token" in token_info:
            print_success(token_info['refresh_token'])
        else:
            print("ERROR: The response from Spotify did NOT include a 'refresh_token'.")
            print(token_info)

    except Exception as e:
        print(f"\nERROR: {e}")
        print("Check if you copied the *entire* URL or if CLIENT_ID/CLIENT_SECRET are correct.")